# 19. Ch 03. Preprocessing - 11. 실습 Subword segmentation

In [ ]:
cut -f2 review.sorted.uniq.refined.tsv > review.sorted.uniq.refined.tsv.text

head -n 2 ./review.sorted.uniq.refined.tsv.text

!
! 다녹아서 왓어요 .. 짜증

-mecab 활요해 세그멘테이션
cat ./review.sorted.uniq.refined.tsv.text | mecab -O wakati | head -n 2

! 
! 다 녹 아서 왓 어요 . . 짜증 

cat ./review.sorted.uniq.refined.tsv.text | mecab -O wakati | python post_tokenize.py ./review.sorted.uniq.refined.tsv.text > review.sorted.uniq.refined.tsv.text.tok

head -n 2 ./review.sorted.uniq.refined.tsv.text.tok

▁!
▁! ▁다 녹 아서 ▁왓 어요 ▁. . ▁짜증

*mecab을 사용한 뒤 생긴 것이 아닌, 원래 띄어쓰기가 있던 자리에 ‘특수문자 _’가 생겼다(일반 _와 다름)
차후 디토크나이제이션 할 때 이게 있는 부분은 빈 칸으로 치환, 없는 부분은 그냥 붙여버리면 됨.

python ./subword-nmt/learn_bpe.py -h

-mecab으로 세그멘테이션된 결과물에 대해서 BPE 알고리즘 학습이 진행
python ./subword-nmt/learn_bpe.py --input ./review.sorted.uniq.refined.tsv.text.tok --output ./model --symbols 30000

-BPE 알고리즘 학습이 진행된 모델 파일을 코퍼스에 적용
python ./subword-nmt/apply_bpe.py --codes ./model < review.sorted.uniq.refined.tsv.text.tok > review.sorted.uniq.refined.tsv.text.tok.bpe

-정규표현식으로 BPE 세그멘테이션 이후 내용 살펴보기
\s[^▁\s]{3,}

-첫번째 컬럼 따로 저장
cut -f1 ./review.sorted.uniq.refined.tsv > review.sorted.uniq.refined.tsv.label

-합치기
paste ./review.sorted.uniq.refined.tsv.label ./review.sorted.uniq.refined.tsv.text.tok.bpe > review.sorted.uniq.refined.tok.bpe.tsv

-스크립트 파일을 가지고 한번에 다 해주기(mecab 토크나이즈 -> 언더라인 추가 -> subword-nmt 세그멘테이션)
./tokenize.sh ./review.sorted.uniq.refined.tsv review.sorted.uniq.refined.tok.bpe.tsv

리뷰는 워낙에 구어체라 mecab이 잘 토크나이즈 못 하긴 한다.(걱정할 필요는 없음) 그래도 하고 나서 서브워드 세그멘테이션 하는 것을 추천.

# 20. Ch 03. Preprocessing - 12. 분절 복원 (detokenization)

In [ ]:
사람들이 읽기 좋은 형태로 만들어주는, 복원
사실 디토크나이제이션은 텍스트 분류 같은 경우는 필요가 없음(결과 값이 확률 값으로 나온다)
NLG에서 필요. 결과를 사람이 봐야 하니까

<토크나이제이션>
1. 영어 원문
There’s currently over a thousand TED Talks on the TED website.
2. tokenization을 수행하고, 기존 띄어쓰기와 구분을 위해 기존 띄어쓰기를 _(U+2581)로 치환
_There’s _currently _over _a _thousand _TED _Talks _on _the _TED _website .
3. 서브워드 세그멘테이션을 수행, 공백 구분 위한 _ 삽입
__There _’s __currently __over __a __thous and __TED __T al ks __on __the __TED __we b site _.

__ 두 개가 붙어 있는 경우는 원래 원문에서 띄어져 있었다
_ 하나가 있으면  2단계(mecab 토크나이제이션)에서 띄어진 것.
띄어져 있는데 아무런 스페셜 캐릭터가 없다면 3단계(서브워드 세그멘테이션)에서 띄어진 것.

<Detokenization>
1.white space를 제거
__There_’s__currently__over__a__thousand__TED__Talks__on__the__TED__website_.
2.__을 white space로 치환
 There_’s currently over a thousand TED Talks on the TED website_.
3._를 제거
 There’s currently over a thousand TED Talks on the TED website.

->sed “s/ //g” | sed “s/ __/ /g” | sed “s/ _//g”

서브워드 세그멘테이션 방식은 언더스코어를 붙이도록 하고 있다.
덕분에 디토크나이제이션을 수월하게 수행할 수 있다.

# 21. Ch 03. Preprocessing - 13. 실습 분절 복원

In [ ]:
기계 번역이나 NLG 같이 문장을 생성해주는 경우에 필요.
사람이 원래 쓰는 형태로 바꿔줌.

import sys

STR = “_”
TWO_STR = “__”

def detokenization(line):
	if TWO_STR in line:
		line = line.strip().replace(‘ ‘, ‘’).replace(TWO_STR, ‘ ‘).replace(STR, ‘’).strip()
	else:
		line = line.strip().replace(‘ ‘, ‘’).replace(STR, ‘ ‘).strip()

	return line
if __name__== “__main__”:
	for line in sys.stdin:
		if line.strip() != “”:
			buf = []
			for token in line.strip().split(‘\t’):
				buf += [detokenization(token)]
			sys.stdout.write(‘\t’.join(buf) + ‘\n’)
		else:
			sts.stdout.write(‘\n’)

python ./detokenizer.py < ./review.sorted.uniq.refined.tok.bpe.tsv > review.sorted.uniq.refined.tok.bpe.detok.tsv

# 22. Ch 03. Preprocessing - 14. 병렬 코퍼스 정렬 시키기(Align Parallel Corpus)

In [ ]:
<Why we need>
-Parallel Corpus?
English / Korean
I love to go to school. <-> 나는 학교에 가는 것을 좋아한다.
I am a doctor. <-> 나는 의사입니다.

-주요 수집 대상
뉴스 기사, 드라마/영화 자막
-대부분의 경우, 문서 단위의 매칭은 돼 있지만, 문장 단위는 돼 있지 않음.
문서 단위의 매칭이 있을 뿐 문장 단위의 매칭은 잘 이뤄져 있지 않다.
뉴스에서 봐도 의역이나 생략이 돼 있을 수 있어서 문장 개수도 맞지 않다.
정렬 작업 필요

<Champollion>
샴폴레옹을 활용해 병렬 코퍼스 정렬
-최초로 이집트 상형문자를 해독한 역사학자
-Perl로 제작 됨(굉장히 오래된 프로그램 중 하나)
-ratio parameter의 역할
소스 언어의 캐릭터 당 target 언어의 캐릭터 비율
-결과 형태:
omitted(버려진다) <=> 1
omitted <=> 2
omitted <=> 3
1 <=> 4
2 <=> 5
3 <=>6
4,5 <=> 7(한국어 문장의 4,5번째를 동시에)
9 <=> omitted

-단어 번역 사전에 기반해 사전을 최대한 만족하는 sentence align을 찾는 방식
align을 할 수 있으면 번역을 할 수 있는 것 아닌가 생각할 수 있지만
번역 필요없이도 어느정도 얼라인 할 수 있다
단어 사전 있어야 한다.

<How to build Word Translation Dictionary>
페이스북에서 공개한 라이브러리 MUSE
지도학습 혹은 준지도학습을 활용해서 시드 단어 사전이 있을 떄 자동으로 어떤 주어진 두 개의 상관 없는 두 언어의 코퍼스를 가지고 자동으로 단어의 번역 사전을 만들어냄.

각 언어에 대해서 워드 임베딩 구함 - 워드 임베딩 서로 같아지게 하다보면 마지막엔 단어의 번역 사전을 찾아낸다는 개념

<예) MUSE Result>
stories <=> 이야기
stories <=> 소설
contact <=> 연락
contact <=> 연락처
green <=> 녹색
green <=> 초록색
green <=> 빨간색(X)

틀린 것도 있지만 큰 리소스를 투입하지 않고도 꽤 괜찮은 결과물(사전)을 얻을 수 있음.
이 사전을 최대한 만족할 수 있는 얼라인을 찾아낸다.

<Procedure to Build Parallel Corpus>
1)Bi-lingual Corpus 정제(예. 노이즈 제거)
2)토크나이제이션 수행(No 서브워드 세그멘테이션)
3)각 언어별 코퍼스에 대해서 워드 임베딩 수행(Fast Text 활용)
4)MUSE를 활용해 word translation dictionary 추출
5)Champollion을 활용해 align 수행

# 23. Ch 03. Preprocessing - 16. TIP 전처리의 중요성, 경험담

In [ ]:
<전처리의 중요성>
-오픈 소스 문화의 확산
요새 같은 경우 소타라고 하는 알고리즘이 한 달에 한 번씩 바뀜
계속된 기술의 발전
좋은 논문들이 아카이브에 굉장히 많이 올라오고 있다.
논문을 내는 순간 소스코드도 공개
너도 나도 최신 성능의 코드를 가지고 있음.

-데이터가 더 큰 자산
성능 차이는 결국 데이터
전처리가 품질의 차이를 만든다.

-반복되고 지치는 업무지만, 소홀히 하면 안 됨.
이상적인 팀 구성은 알고리즘하는 사람, 전처리하는 사람 따로 있어야 한다.
데이터를 전처리하기 위한 휴먼 리소스는 꼭 필요하다.